In [2]:
import spacy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 39.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
#można wykorzystać GloVe, jeśli ktoś woli
nlp = spacy.load('en_core_web_md')

In [4]:
text = """Sweden is a beautiful country, and Stockholm is its capital.
Germany, with its capital Berlin, is known for its rich history.
France has Paris as its capital city.
Italy has Rome as its capital.
Spain is a popular tourist destination, and its capital is Madrid.
Norway is known for its fjords, and Oslo is the capital.
Denmark, with Copenhagen as its capital, is famous for its design and architecture.
Portugal is known for its sunny weather, and Lisbon is the capital.
The Netherlands has Amsterdam as its capital, a city famous for its canals.
Belgium is renowned for its chocolate and waffles, and its capital is Brussels.
Austria's capital is Vienna, known for classical music and art.
Switzerland is famous for its mountains, and Bern is its capital.
Poland, with Warsaw as its capital, has a rich cultural heritage.
The Czech Republic's capital is Prague, known for its old-town charm.
Greece, with Athens as its capital, is the cradle of Western civilization.
Hungary's capital is Budapest, a city split by the Danube River.
Ireland's capital is Dublin, known for its literary history and pubs.
Finland is a land of lakes, and Helsinki is its capital.
Iceland, with Reykjavik as its capital, is known for its volcanic landscapes.
China's capital is Beijing known for red colour.
"""

words = nlp(text)

Napisz funkcję generującą analogie, wykorzystującą podobieństwo cosinusów.
W pierwszym kroku wydobądź nazwy państw i stolic.

In [24]:
countries_capitals = {}
for sentence in words.sents:
    country = None
    capital = None

    for ent in sentence.ents:
        if ent.label_ == "GPE":
            if country is None:  # Jeśli kraj nie został jeszcze znaleziony
                # Sprawdź, czy encja jest krajem na podstawie zależności składniowych
                if any(token.dep_ in ["nsubj", "nsubjpass"] and token.head.text == "is" for token in sentence if token.text == ent.text):
                    country = ent.text
                # Jeśli zdanie zawiera słowo "capital", encja prawdopodobnie jest stolicą
                elif "capital" in sentence.text:
                    capital = ent.text
            else: # Jeśli kraj został już znaleziony, a encja jest "GPE", to prawdopodobnie jest to stolica
                capital = ent.text
                break # Przerwij pętlę, bo znaleźliśmy już stolicę

    if country and capital:
        countries_capitals[country] = capital

print(countries_capitals)

{'Sweden': 'Stockholm', 'Spain': 'Madrid', 'Oslo': 'Norway', 'Lisbon': 'Portugal', 'Belgium': 'Brussels', 'Greece': 'Athens'}


In [28]:
def generate_analogy(question):
    text = nlp(question)

    # Znajdź kraje i stolice w pytaniu
    source_country = None
    source_capital = None
    target_country = None

    for ent in text.ents:
        if ent.label_ == "GPE":
            if source_country is None:
                source_country = ent.text
            elif source_capital is None and ent.text != source_country:
                # Upewnij się, że to nie ten sam kraj co source_country
                source_capital = ent.text
            elif target_country is None and ent.text != source_country and ent.text != source_capital:
                # Upewnij się, że to nie ten sam kraj co source_country i source_capital
                target_country = ent.text
                break  # Przerwij, gdy znaleziono wszystkie trzy elementy

    # Znajdź stolicę dla target_country
    best_capital = None
    max_similarity = -1

    if source_country and source_capital and target_country:
        for country, capital in countries_capitals.items():
            if country == target_country:
                similarity = nlp(source_capital).similarity(nlp(capital))
                if similarity > max_similarity:
                    max_similarity = similarity
                    best_capital = capital

    return best_capital

Dla poniższego zapytania Twój program powinien wygenerować poprawną stolicę - Pekin (Beijing)

In [29]:
question = u"Poland is to Warsaw as Sweden is to WHAT?"
text = nlp(question)
answer = generate_analogy(question)
print(answer)

Stockholm


Uogólnij swoją funckję, tak żeby generowała odpowiedzi dla wszystkich krajów.

In [27]:
#Wykonane w poprzednim kroku, działa dla wwszystkich par państw i stolic z tekstu testowego.

Zaproponuj inny zbiór danych, dla innego, własnego, tematu. Sprawdź jak sobie radzi Twój model.

In [31]:
text = """Stockholm is home to the Karolinska Institute, a world-renowned medical university.
Berlin hosts the Max Planck Institute for Human Development.
Paris is known for the Sorbonne University, a historic center of higher learning.
Rome is the location of the Sapienza University, one of the oldest in Europe.
Madrid is home to the Complutense University, a major research institution.
Oslo houses the University of Oslo, known for contributions to public health research.
Copenhagen is where you'll find the University of Copenhagen, a leading Nordic research university.
Lisbon is home to the University of Lisbon, Portugal’s largest university.
Amsterdam hosts the University of Amsterdam, known for its humanities and social science research.
Brussels is the site of the Université libre de Bruxelles, a prominent Belgian university.
Vienna is where the University of Vienna is located, a historic center for philosophy and law.
Bern is home to the University of Bern, known for space and climate science.
Warsaw hosts the University of Warsaw, a key academic institution in Eastern Europe.
Prague is the location of Charles University, the oldest university in Central Europe.
Athens is home to the National and Kapodistrian University of Athens, focused on classical studies.
Budapest has the Eötvös Loránd University, a hub for scientific research in Hungary.
Dublin is where Trinity College Dublin is located, famous for the Book of Kells.
Helsinki houses the University of Helsinki, Finland’s oldest and largest university.
Reykjavik is home to the University of Iceland, known for its geothermal and environmental research.
Beijing is where Tsinghua University is located, a top institution in science and engineering.
"""

words = nlp(text)

In [33]:
cities_institutes = {}
for sentence in words.sents:
    city = None
    institute = None

    #sprawdzamy po labelach GPE: kraje,miasta oraz ORG: organizacje
    for ent in sentence.ents:
        if ent.label_ == "GPE":
            city = ent.text
        elif ent.label_ == "ORG":
            institute = ent.text

    if city and institute:
        cities_institutes[city] = institute

print(cities_institutes)

{'Stockholm': 'the Karolinska Institute', 'Berlin': 'the Max Planck Institute for Human Development', 'Paris': 'the Sorbonne University', 'Rome': 'the Sapienza University', 'Madrid': 'the Complutense University', 'Oslo': 'the University of Oslo', 'Portugal': 'the University of Lisbon', 'Amsterdam': 'the University of Amsterdam', 'Vienna': 'the University of Vienna', 'Warsaw': 'the University of Warsaw', 'Prague': 'Charles University', 'Athens': 'the National and Kapodistrian University of Athens', 'Hungary': 'the Eötvös Loránd University', 'Dublin': 'Trinity College Dublin', 'Finland': 'the University of Helsinki', 'Beijing': 'Tsinghua University'}


Miasto **Reykjavik** nie działa, nie wykryło go w tekscie

In [34]:
def generate_analogy(question):
    text = nlp(question)

    source_city = None
    source_institute = None
    target_city = None

    #sprawdzamy po labelach GPE: kraje,miasta oraz ORG: organizacje
    for ent in text.ents:
        if ent.label_ == "GPE":
            if source_city is None:
                source_city = ent.text
            elif target_city is None and ent.text != source_city:
                target_city = ent.text
                break
        elif ent.label_ == "ORG" and source_institute is None:
            source_institute = ent.text

    best_institute = None
    max_similarity = -1

    if source_city and source_institute and target_city:
        for city, institute in cities_institutes.items():
            if city == target_city:
                similarity = nlp(source_institute).similarity(nlp(institute))
                if similarity > max_similarity:
                    max_similarity = similarity
                    best_institute = institute

    return best_institute

In [40]:

question = "If Stockholm has the Karolinska Institute, then what is in Paris?"
text = nlp(question)
answer = generate_analogy(question)
print(answer)

None
